<a href="https://colab.research.google.com/github/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/blob/main/Script_Sesi%C3%B3n_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IA para la Ciencia de Datos
## Universidad de los Andes

**Profesor:** Camilo Vega - AI/ML Engineer  
**LinkedIn:** https://www.linkedin.com/in/camilo-vega-169084b1/

---

## Guía: OCR y Extracción de Datos - Análisis Automatizado de Documentos

Este notebook presenta **2 implementaciones prácticas de OCR**:

1. **Tesseract + RAG Database** - Análisis de facturas/recibos con extracción estructurada
2. **Mistral OCR + RAG Documents** - Procesamiento de periódicos con análisis multimodal

### Requisitos
- **APIs:** Groq API token, HuggingFace access
- **GPU:** Opcional para modelos locales
- **Datasets:** Se descargan automáticamente desde HuggingFace

## Configuración APIs
- **Groq API:**
  1. [Crear token](https://console.groq.com/keys)
  2. En Colab: 🔑 Secrets → Agregar `GROQ_KEY` → Pegar tu token



# OCR Treseract +LLM

In [ ]:
# OCR con Tesseract + Visión Groq - Extracción de Datos (con soporte PDF)

"""
ARQUITECTURA DEL SISTEMA OCR + VISION LLM

1. Carga de datos: Ingesta flexible desde HuggingFace datasets o carpetas locales (imágenes + PDF)
2. Análisis de esquema: Primer documento define estructura JSON automáticamente
3. Extracción híbrida: OCR (Tesseract) + Vision LLM (Groq) en paralelo
4. Mapeo inteligente: LLM estructura datos según esquema aprendido
5. Procesamiento batch: Aplica pipeline a todos los documentos
6. Salida JSON: Resultados estructurados con metadatos y validación
"""

# Instalaciones necesarias
!pip install tesseract pytesseract Pillow datasets groq pdf2image -q
!apt update && apt install tesseract-ocr poppler-utils -y -q

import json
import pytesseract
import base64
import io
import os
import glob
from pathlib import Path
from PIL import Image
from datasets import load_dataset
from groq import Groq
from google.colab import userdata
from pdf2image import convert_from_path
import tempfile

# HIPERPARÁMETROS CONFIGURABLES
USE_LOCAL_FOLDER = False  # True: usar carpeta local, False: usar dataset
CARPETA_FACTURAS = "/content/facturas"  # Carpeta con facturas propias
DATASET_NAME = "v2run/invoices-donut-data-v1"  # Dataset de facturas HF
NUM_DOCUMENTOS = 10  # Número de documentos a procesar en fase 2
FORMATOS_IMAGEN = ["*.png", "*.jpg", "*.jpeg", "*.tiff", "*.bmp"]
FORMATOS_PDF = ["*.pdf"]
PDF_DPI = 200  # Resolución para conversión PDF a imagen

# Configuración
client = Groq(api_key=userdata.get('GROQ_KEY'))

def get_document_files(carpeta, num_max):
    """Obtiene lista de archivos de imagen y PDF de la carpeta"""
    archivos = []
    # Imágenes
    for formato in FORMATOS_IMAGEN:
        archivos.extend(glob.glob(os.path.join(carpeta, formato)))
        archivos.extend(glob.glob(os.path.join(carpeta, formato.upper())))
    # PDFs
    for formato in FORMATOS_PDF:
        archivos.extend(glob.glob(os.path.join(carpeta, formato)))
        archivos.extend(glob.glob(os.path.join(carpeta, formato.upper())))
    return sorted(archivos)[:num_max + 1]  # +1 para el ejemplo

def pdf_to_image(pdf_path, page_num=0):
    """Convierte página específica de PDF a imagen PIL"""
    try:
        # Convertir PDF a imágenes
        images = convert_from_path(pdf_path, dpi=PDF_DPI, first_page=page_num+1, last_page=page_num+1)
        if images:
            return images[0]
        else:
            raise Exception("No se pudo convertir PDF a imagen")
    except Exception as e:
        print(f"Error convirtiendo PDF {pdf_path}: {e}")
        return None

def load_local_documents(carpeta, num_docs):
    """Carga documentos (imágenes y PDFs) desde carpeta local"""
    archivos = get_document_files(carpeta, num_docs)

    if not archivos:
        raise Exception(f"No se encontraron documentos en: {carpeta}")

    documents_data = []
    for archivo in archivos:
        file_path = Path(archivo)
        file_ext = file_path.suffix.lower()

        try:
            if file_ext == '.pdf':
                # Convertir primera página del PDF a imagen
                img = pdf_to_image(archivo, page_num=0)
                if img is None:
                    print(f"Saltando PDF problemático: {file_path.name}")
                    continue
                document_type = "pdf"
            else:
                # Cargar imagen directamente
                img = Image.open(archivo)
                document_type = "image"

            documents_data.append({
                'image': img,
                'filename': file_path.name,
                'type': document_type,
                'original_path': archivo
            })

        except Exception as e:
            print(f"Error procesando {file_path.name}: {e}")
            continue

    return documents_data

def image_to_base64(image):
    """Convierte imagen PIL a base64"""
    buffer = io.BytesIO()
    image.save(buffer, format='PNG')
    img_str = base64.b64encode(buffer.getvalue()).decode()
    return f"data:image/png;base64,{img_str}"

def analyze_example_structure(ejemplo_imagen):
    """Analiza la imagen de ejemplo para definir la estructura JSON"""
    try:
        image_base64 = image_to_base64(ejemplo_imagen)

        print("Analizando documento de ejemplo para definir estructura...")

        completion = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": """Analiza esta imagen de factura/documento y define una estructura JSON que capture TODOS los campos visibles.

Crea un formato JSON completo que incluya:
- Números de identificación
- Fechas
- Información de cliente/proveedor
- Productos/servicios con precios
- Totales y subtotales

Responde SOLO con la estructura JSON:"""
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": image_base64}
                        }
                    ]
                }
            ],
            temperature=0.1,
            max_tokens=600
        )

        result = completion.choices[0].message.content.strip()
        result = result.replace('```json', '').replace('```', '').strip()

        estructura = json.loads(result)
        print("Estructura definida:")
        print(json.dumps(estructura, indent=2, ensure_ascii=False))

        return estructura

    except Exception as e:
        print(f"Error analizando ejemplo: {e}")
        return {
            "invoice_number": "",
            "date": "",
            "customer_name": "",
            "total_amount": 0
        }

def extract_text_tesseract(image):
    """Extrae texto con Tesseract"""
    config = '--oem 3 --psm 6'
    return pytesseract.image_to_string(image, config=config).strip()

def analyze_with_vision(image, estructura_aprendida):
    """Analiza documentos con el modelo de visión correcto"""
    try:
        image_base64 = image_to_base64(image)
        campos_estructura = json.dumps(estructura_aprendida, indent=2, ensure_ascii=False)

        completion = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": f"""Extrae información de esta imagen de factura usando EXACTAMENTE esta estructura JSON:

{campos_estructura}

REGLAS ESTRICTAS:
- Usa los mismos nombres de campos exactamente
- Si no encuentras un campo, usa null (sin comillas)
- TODOS los strings deben estar entre comillas dobles
- NO uses comillas simples
- CIERRA todos los corchetes y llaves
- NO incluyas texto adicional fuera del JSON
- VERIFICA que el JSON esté completo y válido

Responde ÚNICAMENTE con JSON válido, nada más:"""
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": image_base64}
                        }
                    ]
                }
            ],
            temperature=0.05,
            max_tokens=800
        )

        result = completion.choices[0].message.content.strip()

        # Limpieza más agresiva
        result = result.replace('```json', '').replace('```', '').strip()

        # Remover texto antes y después del JSON si existe
        start_idx = result.find('{')
        end_idx = result.rfind('}')

        if start_idx != -1 and end_idx != -1:
            result = result[start_idx:end_idx + 1]

        # Intentar parsear
        parsed_result = json.loads(result)
        return parsed_result

    except json.JSONDecodeError as e:
        return {"error": f"JSON malformado: {str(e)}", "raw_response": result[:200]}
    except Exception as e:
        return {"error": f"Error: {str(e)}"}

# CARGA DE DATOS
print("="*50)
print("CARGANDO DATOS")
print("="*50)

if USE_LOCAL_FOLDER:
    print(f"Usando carpeta local: {CARPETA_FACTURAS}")
    print(f"Formatos soportados: {', '.join(FORMATOS_IMAGEN + FORMATOS_PDF)}")
    print(f"DPI para conversión PDF: {PDF_DPI}")

    try:
        documents_data = load_local_documents(CARPETA_FACTURAS, NUM_DOCUMENTOS)
        print(f"Encontrados {len(documents_data)} documentos")

        # Mostrar tipos de documentos encontrados
        tipos = {}
        for doc in documents_data:
            tipos[doc['type']] = tipos.get(doc['type'], 0) + 1
        print(f"Tipos: {tipos}")

        # Separar ejemplo y documentos a procesar
        ejemplo_doc = documents_data[0]
        documentos_procesar = documents_data[1:NUM_DOCUMENTOS + 1]

        print(f"Usando {ejemplo_doc['filename']} ({ejemplo_doc['type']}) como ejemplo")
        print(f"Procesando {len(documentos_procesar)} documentos")

        source_info = f"Carpeta: {CARPETA_FACTURAS}"

    except Exception as e:
        print(f"Error cargando carpeta local: {e}")
        print("Fallback a dataset...")
        USE_LOCAL_FOLDER = False

if not USE_LOCAL_FOLDER:
    print(f"Usando dataset: {DATASET_NAME}")
    print(f"Documentos a procesar: {NUM_DOCUMENTOS}")

    try:
        dataset = load_dataset(DATASET_NAME, split=f"train[:{NUM_DOCUMENTOS + 1}]")
        documents = list(dataset)
        print(f"Dataset cargado: {len(documents)} documentos")

        # Separar ejemplo y documentos a procesar
        ejemplo_doc = {'image': documents[0]['image'], 'filename': 'dataset_ejemplo', 'type': 'image'}
        documentos_procesar = [{'image': doc['image'], 'filename': f'dataset_doc_{i+2}', 'type': 'image'}
                             for i, doc in enumerate(documents[1:NUM_DOCUMENTOS + 1])]

        print(f"Usando documento 1 como ejemplo")
        print(f"Procesando documentos 2-{len(documentos_procesar) + 1}")

        source_info = f"Dataset: {DATASET_NAME}"

    except Exception as e:
        print(f"Error cargando dataset: {e}")
        exit()

# EJECUCIÓN PRINCIPAL
print(f"\n{'='*50}")
print("FASE 1: ANÁLISIS DE EJEMPLO")
print("="*50)

estructura_json = analyze_example_structure(ejemplo_doc['image'])

print(f"\n{'='*50}")
print("FASE 2: PROCESAMIENTO DOCUMENTOS")
print("="*50)

try:
    print(f"Procesando {len(documentos_procesar)} documentos")

    results = []

    for i, doc in enumerate(documentos_procesar):
        print(f"\nDocumento {i+1} ({doc['filename']}) - {doc['type'].upper()}:")

        try:
            image = doc['image']

            # OCR
            ocr_text = extract_text_tesseract(image)
            print(f"  OCR: {len(ocr_text)} caracteres")

            # Visión
            print(f"  Analizando con visión...")
            vision_data = analyze_with_vision(image, estructura_json)

            result = {
                "id": i+1,
                "filename": doc['filename'],
                "document_type": doc['type'],
                "ocr_length": len(ocr_text),
                "ocr_sample": ocr_text[:200] + "..." if len(ocr_text) > 200 else ocr_text,
                "vision_analysis": vision_data,
                "status": "processed" if not vision_data.get("error") else "error"
            }

            results.append(result)
            print(f"  Estado: {result['status']}")

            if result['status'] == 'error':
                print(f"  Error: {vision_data.get('error', 'Desconocido')}")
            else:
                campos = len([v for v in vision_data.values() if v and str(v).lower() != "null"])
                print(f"  Campos detectados: {campos}")

        except Exception as e:
            print(f"  Error general: {e}")
            results.append({
                "id": i+1,
                "filename": doc['filename'],
                "document_type": doc.get('type', 'unknown'),
                "status": "failed",
                "error": str(e)
            })

    # Resultados finales
    print(f"\n{'='*50}")
    print("RESULTADOS")
    print("="*50)

    successful = [r for r in results if r.get('status') == 'processed']
    print(f"Exitosos: {len(successful)}/{len(results)}")

    # Estadísticas por tipo
    tipos_procesados = {}
    for r in results:
        doc_type = r.get('document_type', 'unknown')
        if doc_type not in tipos_procesados:
            tipos_procesados[doc_type] = {'total': 0, 'exitosos': 0}
        tipos_procesados[doc_type]['total'] += 1
        if r.get('status') == 'processed':
            tipos_procesados[doc_type]['exitosos'] += 1

    print("\nEstadísticas por tipo:")
    for tipo, stats in tipos_procesados.items():
        print(f"  {tipo.upper()}: {stats['exitosos']}/{stats['total']}")

    if successful:
        print("\nMejor ejemplo procesado:")
        best = max(successful,
                  key=lambda x: len(str(x.get('vision_analysis', {}))))
        print(f"Archivo: {best['filename']} ({best['document_type']})")
        print(json.dumps(best['vision_analysis'], indent=2, ensure_ascii=False))

        print(f"\nTexto OCR (muestra):")
        print(f"{'-'*30}")
        print(best.get('ocr_sample', 'Sin texto'))

    # Guardar
    with open('resultados_ocr.json', 'w', encoding='utf-8') as f:
        json.dump({
            "configuracion": {
                "fuente": source_info,
                "usar_carpeta_local": USE_LOCAL_FOLDER,
                "num_documentos": NUM_DOCUMENTOS,
                "ejemplo_usado": ejemplo_doc['filename'],
                "ejemplo_tipo": ejemplo_doc['type'],
                "pdf_dpi": PDF_DPI,
                "formatos_soportados": FORMATOS_IMAGEN + FORMATOS_PDF
            },
            "estructura_aprendida": estructura_json,
            "documentos_procesados": results,
            "estadisticas_tipos": tipos_procesados
        }, f, indent=2, ensure_ascii=False, default=str)
    print("\nGuardado en: resultados_ocr.json")

except Exception as e:
    print(f"Error: {e}")

In [ ]:
# EJEMPLO DE EXTRACCIÓN CON DOCUMENTOS ESCANEADOS

import requests
from IPython.display import display

print("\n--- PROCESAMIENTO EJEMPLO MANUAL ---")

# Descargar factura desde GitHub
FACTURA_URL = "https://github.com/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/raw/main/Ejemplo%20Factura%20Escaneada.jpg"

response = requests.get(FACTURA_URL, timeout=30)
with open("/content/Ejemplo Factura Escaneada.jpg", 'wb') as f:
    f.write(response.content)

EJEMPLO_DOCUMENTO = "/content/Ejemplo Factura Escaneada.jpg"

if os.path.exists(EJEMPLO_DOCUMENTO):
    # Cargar documento (PDF o imagen)
    file_ext = Path(EJEMPLO_DOCUMENTO).suffix.lower()
    if file_ext == '.pdf':
        imagen = pdf_to_image(EJEMPLO_DOCUMENTO, page_num=0)
    else:
        imagen = Image.open(EJEMPLO_DOCUMENTO)

    # Mostrar imagen
    display(imagen.resize((imagen.width // 2, imagen.height // 2)))

    # Procesar
    ocr_text = extract_text_tesseract(imagen)
    estructura = analyze_example_structure(imagen)
    resultado = analyze_with_vision(imagen, estructura)

    # Mostrar
    print(f"Archivo: {Path(EJEMPLO_DOCUMENTO).name}")

else:
    print(f"Archivo no encontrado: {EJEMPLO_DOCUMENTO}")

#OCR Mistral

In [ ]:
# OCR Mistral - Procesamiento de PDFs locales

"""
ARQUITECTURA DEL SISTEMA OCR MISTRAL - PROCESAMIENTO DE PDFs

1. Configuración de entorno: Inicialización de cliente Mistral y carpetas de trabajo
2. Ingesta de documentos: Descarga automática + subida manual de archivos PDF
3. OCR avanzado: Extracción de texto, markdown e imágenes con Mistral OCR API
4. Análisis inteligente: LLM procesa contenido y genera metadatos estructurados
5. Extracción multimedia: Imágenes embebidas guardadas como archivos independientes
6. Persistencia múltiple: Salida en formatos Markdown y JSON con resumen
7. Visualización: Display de imágenes y fragmentos importantes
8. Documentación Mistral OCR:https://mistral.ai/news/mistral-ocr
"""

!pip install mistralai pillow -q

import json
import base64
import os
import requests
from pathlib import Path
from mistralai import Mistral, DocumentURLChunk, ImageURLChunk, TextChunk
from google.colab import userdata, files
from PIL import Image
from IPython.display import display, Markdown

# Configuración
OCR_FOLDER = '/content/archivos_ocr'
RESULTS_FOLDER = '/content/resultados_ocr'
PAPER_URL = "https://github.com/CamiloVga/Curso-IA-Para-Ciencia-de-Datos/raw/main/Paper%20Attention%20Is%20All%20You%20Need.pdf"
PAPER_FILENAME = "Paper_Attention_Is_All_You_Need.pdf"

# Setup cliente Mistral
client = Mistral(api_key=userdata.get('MISTRAL'))

def create_folders():
    """Crea carpetas necesarias"""
    os.makedirs(OCR_FOLDER, exist_ok=True)
    os.makedirs(RESULTS_FOLDER, exist_ok=True)

def download_paper():
    """Descarga paper desde GitHub"""
    try:
        create_folders()
        response = requests.get(PAPER_URL, timeout=30)
        response.raise_for_status()

        paper_path = os.path.join(OCR_FOLDER, PAPER_FILENAME)
        with open(paper_path, 'wb') as f:
            f.write(response.content)

        return True
    except Exception as e:
        print(f"Error descargando: {str(e)}")
        return False

def upload_files():
    """Permite subir archivos adicionales"""
    uploaded = files.upload()

    for filename, content in uploaded.items():
        dest_path = os.path.join(OCR_FOLDER, filename)
        with open(dest_path, 'wb') as f:
            f.write(content)

    return list(uploaded.keys())

def list_pdf_files():
    """Lista archivos PDF en carpeta"""
    if not os.path.exists(OCR_FOLDER):
        return []

    pdf_files = [f for f in os.listdir(OCR_FOLDER) if f.lower().endswith('.pdf')]
    return sorted(pdf_files)

def process_pdf_with_mistral(pdf_filename):
    """Procesa PDF usando Mistral OCR"""
    try:
        pdf_path = Path(os.path.join(OCR_FOLDER, pdf_filename))
        if not pdf_path.is_file():
            raise FileNotFoundError(f"Archivo {pdf_filename} no encontrado")

        print(f"Subiendo {pdf_filename} a Mistral OCR...")

        # Subir PDF al servicio OCR
        uploaded_file = client.files.upload(
            file={
                "file_name": pdf_path.stem,
                "content": pdf_path.read_bytes(),
            },
            purpose="ocr",
        )

        # Obtener URL firmada
        signed_url = client.files.get_signed_url(file_id=uploaded_file.id, expiry=1)

        print(f"Procesando con OCR...")

        # Procesar con OCR
        pdf_response = client.ocr.process(
            document=DocumentURLChunk(document_url=signed_url.url),
            model="mistral-ocr-latest",
            include_image_base64=True
        )

        print(f"OCR completado - {len(pdf_response.pages)} páginas procesadas")
        return pdf_response

    except Exception as e:
        print(f"Error procesando {pdf_filename}: {str(e)}")
        return None

def extract_text_and_markdown(ocr_response):
    """Extrae texto y markdown del OCR"""
    combined_text = ""
    combined_markdown = ""

    if ocr_response and hasattr(ocr_response, 'pages'):
        for page in ocr_response.pages:
            if hasattr(page, 'markdown') and page.markdown:
                combined_text += page.markdown + "\n\n"
                combined_markdown += page.markdown + "\n\n"

    return combined_text.strip(), combined_markdown.strip()

def analyze_with_chat_model(text, filename):
    """Analiza texto con modelo de chat"""
    try:
        response = client.chat.complete(
            model="pixtral-12b-latest",
            messages=[{
                "role": "user",
                "content": f"""Analiza este documento y extrae información estructurada en JSON:

Archivo: {filename}
Texto: {text[:3000]}

Extrae y devuelve JSON con:
- title: título del documento
- document_type: tipo de documento
- main_topics: lista de temas principales
- key_points: puntos clave
- language: idioma detectado
- summary: resumen breve

Responde solo con JSON válido:"""
            }],
            temperature=0.1,
            max_tokens=800
        )

        result_text = response.choices[0].message.content.strip()
        result_text = result_text.replace('```json', '').replace('```', '').strip()

        # Extraer JSON
        start = result_text.find('{')
        end = result_text.rfind('}')
        if start != -1 and end != -1:
            result_text = result_text[start:end + 1]

        return json.loads(result_text)

    except Exception as e:
        return {
            "title": filename,
            "document_type": "documento",
            "main_topics": ["contenido general"],
            "key_points": ["información extraída por OCR"],
            "language": "unknown",
            "summary": text[:200] + "..." if len(text) > 200 else text
        }

def extract_images_from_ocr(ocr_response, base_filename):
    """Extrae imágenes del OCR y las muestra"""
    extracted_images = []

    if not ocr_response or not hasattr(ocr_response, 'pages'):
        return extracted_images

    for page_idx, page in enumerate(ocr_response.pages):
        if hasattr(page, 'images'):
            for img_idx, img in enumerate(page.images):
                if hasattr(img, 'image_base64') and img.image_base64:
                    try:
                        # Procesar datos base64
                        base64_data = img.image_base64
                        if "," in base64_data:
                            base64_data = base64_data.split(",", 1)[1]

                        # Guardar imagen
                        img_filename = f"{base_filename}_page{page_idx+1}_img{img_idx+1}.jpg"
                        img_path = os.path.join(RESULTS_FOLDER, img_filename)

                        with open(img_path, 'wb') as f:
                            f.write(base64.b64decode(base64_data))

                        # Mostrar imagen extraída
                        print(f"\nImagen extraída: Página {page_idx+1}, Imagen {img_idx+1}")
                        extracted_img = Image.open(img_path)
                        display(extracted_img.resize((min(400, extracted_img.width),
                                                    min(300, extracted_img.height))))

                        extracted_images.append(img_filename)

                    except Exception as e:
                        print(f"Error guardando imagen: {e}")

    return extracted_images

def display_content_preview(text, structured_data):
    """Muestra preview del contenido extraído"""
    print("\n--- PREVIEW DEL CONTENIDO ---")

    # Mostrar fragmento del texto
    preview_text = text[:800] + "..." if len(text) > 800 else text
    print(f"Fragmento del texto extraído ({len(text)} caracteres total):")
    print("-" * 50)
    print(preview_text)
    print("-" * 50)

    # Mostrar análisis estructurado
    print("\nAnálisis estructurado:")
    display(Markdown(f"""
### {structured_data.get('title', 'Sin título')}
**Tipo:** {structured_data.get('document_type', 'N/A')}
**Idioma:** {structured_data.get('language', 'N/A')}

**Resumen:**
{structured_data.get('summary', 'No disponible')}

**Temas principales:**
{', '.join(structured_data.get('main_topics', []))}
"""))

def save_results(filename, text, markdown, structured_data, extracted_images):
    """Guarda resultados en Markdown y JSON (sin TXT)"""
    base_name = os.path.splitext(filename)[0]

    # Markdown estructurado
    title = structured_data.get('title', 'Documento sin título')
    markdown_content = f"""# {title}

Archivo: {filename}
Tipo: {structured_data.get('document_type', 'Documento')}
Idioma: {structured_data.get('language', 'No detectado')}

## Resumen
{structured_data.get('summary', 'No disponible')}

## Temas Principales
{chr(10).join(f"- {topic}" for topic in structured_data.get('main_topics', []))}

## Puntos Clave
{chr(10).join(f"- {point}" for point in structured_data.get('key_points', []))}

## Imágenes Extraídas
{chr(10).join(f"- {img}" for img in extracted_images) if extracted_images else "No se extrajeron imágenes"}

## Contenido Completo
{text}

## Markdown Original
{markdown}
"""

    markdown_file = os.path.join(RESULTS_FOLDER, f"{base_name}.md")
    with open(markdown_file, 'w', encoding='utf-8') as f:
        f.write(markdown_content)

    # JSON estructurado
    json_data = {
        "filename": filename,
        "extracted_images": extracted_images,
        "text_length": len(text),
        "structured_analysis": structured_data,
        "full_text": text,
        "markdown": markdown
    }

    json_file = os.path.join(RESULTS_FOLDER, f"{base_name}.json")
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)

    return {
        "markdown_file": markdown_file,
        "json_file": json_file,
        "images": extracted_images
    }

def process_all_pdfs():
    """Procesa todos los PDFs en carpeta"""
    pdf_files = list_pdf_files()

    if not pdf_files:
        print("No se encontraron archivos PDF")
        return []

    results = []

    for i, pdf_file in enumerate(pdf_files):
        print(f"\n{'='*60}")
        print(f"Procesando {i+1}/{len(pdf_files)}: {pdf_file}")
        print(f"{'='*60}")

        try:
            # Procesar con OCR
            ocr_response = process_pdf_with_mistral(pdf_file)
            if not ocr_response:
                results.append({"filename": pdf_file, "status": "error", "error": "No OCR response"})
                continue

            # Extraer contenido
            text, markdown = extract_text_and_markdown(ocr_response)
            structured_data = analyze_with_chat_model(text, pdf_file)

            # Mostrar preview del contenido
            display_content_preview(text, structured_data)

            # Extraer imágenes (con display automático)
            base_name = os.path.splitext(pdf_file)[0]
            extracted_images = extract_images_from_ocr(ocr_response, base_name)

            # Guardar resultados
            saved_files = save_results(pdf_file, text, markdown, structured_data, extracted_images)

            results.append({
                "filename": pdf_file,
                "status": "success",
                "text_length": len(text),
                "images_extracted": len(extracted_images),
                "structured_data": structured_data,
                "saved_files": saved_files
            })

        except Exception as e:
            print(f"Error procesando {pdf_file}: {str(e)}")
            results.append({"filename": pdf_file, "status": "error", "error": str(e)})

    return results

def main():
    """Función principal"""
    print("OCR MISTRAL - PROCESADOR DE PDFs CON VISUALIZACIONES")

    # Crear carpetas
    create_folders()

    # Descargar paper
    download_success = download_paper()
    if not download_success:
        print("Descarga automática falló")

    # Verificar archivos
    existing_files = list_pdf_files()
    if not existing_files:
        print("No hay archivos PDF. Ejecuta upload_files() para subir archivos.")
        return []

    # Procesar PDFs
    results = process_all_pdfs()

    # Resumen
    successful = [r for r in results if r['status'] == 'success']
    errors = [r for r in results if r['status'] == 'error']

    print(f"\n{'='*60}")
    print("RESUMEN FINAL")
    print(f"{'='*60}")
    print(f"Procesados: {len(successful)}/{len(results)}")
    print(f"Errores: {len(errors)}")

    if successful:
        total_images = sum(r.get('images_extracted', 0) for r in successful)
        print(f"Imágenes extraídas y mostradas: {total_images}")

    # Guardar resumen
    summary_file = os.path.join(RESULTS_FOLDER, "resumen_procesamiento.json")
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump({
            "total_processed": len(results),
            "successful": len(successful),
            "errors": len(errors),
            "results": results
        }, f, indent=2, ensure_ascii=False)

    return results

# Funciones auxiliares
def download_paper_only():
    return download_paper()

def process_paper_only():
    paper_path = os.path.join(OCR_FOLDER, PAPER_FILENAME)
    if not os.path.exists(paper_path):
        print("Paper no encontrado. Ejecuta download_paper_only() primero.")
        return None

    return process_pdf_with_mistral(PAPER_FILENAME)

# Ejecutar
if __name__ == "__main__":
    main()

In [ ]:
# EJEMPLO DE EXTRACCIÓN CON DOCUMENTOS ESCANEADOS

import base64
import requests
from PIL import Image
from IPython.display import display

# Descargar y procesar imagen
RECEIPT_URL = "https://raw.githubusercontent.com/mistralai/cookbook/refs/heads/main/mistral/ocr/receipt.png"
receipt_path = os.path.join(OCR_FOLDER, "receipt.png")

# Descarga
response = requests.get(RECEIPT_URL, timeout=30)
with open(receipt_path, 'wb') as f:
    f.write(response.content)

# Mostrar imagen
img = Image.open(receipt_path)
display(img.resize((img.width // 3, img.height // 3)))

# OCR
encoded = base64.b64encode(Path(receipt_path).read_bytes()).decode()
base64_data_url = f"data:image/jpeg;base64,{encoded}"

image_response = client.ocr.process(
    document=ImageURLChunk(image_url=base64_data_url),
    model="mistral-ocr-latest"
)

# Mostrar resultado
ocr_text = image_response.pages[0].markdown
print("Contenido extraído:")
print(ocr_text)

# Análisis estructurado
chat_response = client.chat.complete(
    model="pixtral-12b-latest",
    messages=[{
        "role": "user",
        "content": [
            ImageURLChunk(image_url=base64_data_url),
            TextChunk(text=f"OCR: {ocr_text}\nConvierte a JSON estructurado:")
        ]
    }],
    response_format={"type": "json_object"},
    temperature=0
)

# Resultado final
result = json.loads(chat_response.choices[0].message.content)
print("\nAnálisis estructurado:")
print(json.dumps(result, indent=2, ensure_ascii=False))

# Guardar
with open(os.path.join(RESULTS_FOLDER, "receipt_analysis.json"), 'w') as f:
    json.dump({"ocr": ocr_text, "analysis": result}, f, indent=2, ensure_ascii=False)

print("Completado")